In [2]:
import json
import numpy as np
import torch
from torch_geometric.data import Data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from transformers import pipeline


In [3]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [4]:
with open('edinburgh-keywords_train.json', 'r') as f:
    train_data = json.load(f)

keywords = list(train_data['np2count'].keys())

keyword_set = set(keywords)

In [5]:
def extract_users(info):
    l_user, user2kw = [], []
    for ii in info:
        lus = info[ii]
        for u in lus:
            if u not in l_user:
                l_user.append(u)
                user2kw.append([])
            idx = l_user.index(u)
            user2kw[idx].append(ii)
    return l_user, user2kw

In [6]:
train_users, train_users2kw = extract_users(train_data['np2users'])


In [7]:
restaurant_set = set()
listres = []
for kw in train_data['np2rests'].keys():
    listres.extend(train_data['np2rests'][kw].keys())
restaurant_set = set(listres)

keyword_set = list(keyword_set)
restaurant_set = list(restaurant_set)
restaurants = len(listres)
num_keywords = len(keyword_set)
num_restaurants = len(restaurant_set)
a = np.zeros((num_keywords, num_restaurants))

In [8]:
for kw in train_data['np2rests'].keys():
    for res in train_data['np2rests'][kw].keys():
        idx_kw = keyword_set.index(kw)
        idx_res = restaurant_set.index(res)
        a[idx_kw][idx_res] = 1

In [9]:
keyword_embeddings = model.encode(list(keyword_set))

In [10]:
with open('edinburgh-keywords_test.json', 'r') as r:
    test_data = json.load(r)

user_keywords = list(test_data['np2reviews'].keys())
user_keywords_list = list(user_keywords)

In [11]:
test_users, test_users2kw = extract_users(test_data['np2users'])


In [12]:
test_keywords = [kw for sublist in test_users2kw for kw in sublist]
test_keyword_embeddings = model.encode(test_keywords)

similarity_scores = cosine_similarity(test_keyword_embeddings, keyword_embeddings)

In [13]:
filtered_keywords = []
for i, user_kw in enumerate(test_users2kw):
    updated_user_kw = []
    for kw in user_kw:
        if kw not in keyword_set:
            test_idx = test_keywords.index(kw)
            sim_scores = similarity_scores[test_idx]

            best_match_idx = np.argmax(sim_scores)
            best_match_keyword = keyword_set[best_match_idx]

            updated_user_kw.append(best_match_keyword)
        else:
            updated_user_kw.append(kw)

    filtered_keywords.append(updated_user_kw)

test_users2kw = filtered_keywords

In [14]:
results = []
for kw in test_users2kw:
    t = np.zeros((1, len(keyword_set)))
    keywords = kw[:10]
    for keys in keywords:
        if keys in keyword_set:
            idx_kw = keyword_set.index(keys)
            t[0][idx_kw] = 1
    R = np.dot(t, a)
    result = np.argsort(R[0])[::-1][:10]
    results.append(result)

In [1]:
# if __name__ == "__main__":
#         for i, result in enumerate(results):
#             restaurant_names = [restaurant_set[idx] for idx in result]
#             print(f"The result for user {i} is: {restaurant_names}")

SỬ DỤNG LLM ĐỂ RE-RANK

In [34]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


In [1]:
llm_pipeline = pipeline("text-generation", model="gpt2")

def re_rank_candidates(user_id, candidate_restaurants, user_keywords):
    """
    Sử dụng LLM để re-rank các candidate restaurants dựa trên từ khóa của người dùng.

    Args:
        user_id (str): Mã định danh của người dùng.
        candidate_restaurants (list): Danh sách tên restaurant được candidate.
        user_keywords (list): Danh sách từ khóa liên quan đến sở thích của người dùng.

    Returns:
        list: Danh sách restaurant được xếp hạng lại.
    """
    prompt = (
        f"Người dùng có sở thích: {', '.join(user_keywords[:5])}. "  
        f"Với các candidate restaurants sau: {', '.join(candidate_restaurants[:5])}. "  
        "Hãy xếp hạng lại các restaurant theo mức độ phù hợp với sở thích của người dùng theo thứ tự giảm dần và chỉ in ra danh sách tên các restaurant, cách nhau bằng dấu phẩy."
    )
    
    generated = llm_pipeline(prompt, max_new_tokens=200, do_sample=False)[0]['generated_text']
    
    output = generated[len(prompt):].strip()
    
    re_ranked = [restaurant.strip() for restaurant in output.split(',')]
    
    valid_re_ranked = [r for r in re_ranked if r in candidate_restaurants]
    if len(valid_re_ranked) == 0:
        valid_re_ranked = candidate_restaurants
    return valid_re_ranked

final_results = []
for idx, (user, candidate_indices) in enumerate(zip(test_users, results)):
    candidate_restaurants = [restaurant_set[i] for i in candidate_indices]
    user_kw = test_users2kw[idx]
    re_ranked = re_rank_candidates(user, candidate_restaurants, user_kw)
    final_results.append(re_ranked)
    print(f"Re-ranked results for user {user}: {re_ranked}")

NameError: name 'pipeline' is not defined

In [ ]:
import json

output_data = {}

for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
    user_data = {}
    
    user_keywords = test_users2kw[idx]
    
    candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]
    
    positions = [str(i) for i in restaurant_indices]

    re_ranked = re_rank_candidates(user, candidate_restaurants, user_keywords)
    
    user_data["kw"] = user_keywords[:10]
    user_data["candidate"] = re_ranked[:10]
    user_data["positions"] = positions[:10]
    
    output_data[user] = user_data

json_file_path = './result/results.json'

with open(json_file_path, mode="w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print(f"\nResults saved to: {json_file_path}")



Results saved to: ./result/results.json


In [23]:
import json


output_data = {}

for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
    user_data = {}

    user_keywords = test_users2kw[idx]

    candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]

    re_ranked_restaurants = re_ranked

    positions = [str(i) for i in restaurant_indices]  # 1-based index

    user_data["kw"] = user_keywords[:10]  
    user_data["candidate"] = re_ranked_restaurants[:10] 
    user_data["positions"] = positions[:10]  
    output_data[user] = user_data

file_path='./result/results(afterLLM).json'

with open(file_path, mode="w", encoding="utf-8") as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

print(f"Results saved to: {file_path}")


Results saved to: ./result/results(afterLLM).json
